In [ ]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from scipy.stats import pearsonr
from tqdm import tqdm
from scipy import ndimage
from IPython.display import clear_output

Using TensorFlow backend.


In [ ]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 32
NUM_CLASSES = 10
NUM_CHANNELS = 3
MODEL_NAME = "CIFAR_rotation"
VOTING = 'HARD'
PATH = "/content/drive/My Drive/NC/"
MODEL_ADDITION_DELTA = 0.01
MODEL_ADDITION_PATIENCE = 3
NR_OF_RUNS = 10

# Preprocess

In [ ]:
def preprocess(imgs):
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [ ]:
(x_train_val, y_train_val), (x_test, y_test) = cifar10.load_data()

#x_train_val = preprocess(x_train_val)
#x_test = preprocess(x_test)

print('x_train shape:', x_train_val.shape)
print(x_train_val.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
# Convert class vectors to binary class matrices.
y_train_val = keras.utils.to_categorical(y_train_val, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
x_train_val = x_train_val.astype('float32')
x_test = x_test.astype('float32')
x_train_val /= 255
x_test /= 255

# Model

In [ ]:
def CIFARmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=2)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=128, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters=10, kernel_size=(1,1),strides=(1,1), padding='valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

#Predict


In [ ]:
def hard_voting(models, X):
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(X), axis=1))

    prediction = np.transpose(predictions)
    prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

    return prediction

def soft_voting(models, X):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in models:
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return prediction

def predict(models, X, Y,voting = 'hard'):
    
    if voting == "soft":
      prediction = soft_voting(models, X)
    elif voting == "hard":
      prediction = hard_voting(models, X)
    else:
      raise ValueError(f"Voting mechanism: {VOTING} not supported")
    score = accuracy_score(prediction, np.argmax(Y, axis=1))
    print(score)
    return score

#Augmentation 


In [ ]:
def rotate_image(image):
    if np.random.rand() < 0.2:
        angles = np.linspace(1,10,10)
        rotation_angle = np.random.choice(angles)
        if np.random.rand() < 0.5:
            image = ndimage.rotate(image, rotation_angle, reshape = False)
        else:
            image = ndimage.rotate(image, -rotation_angle, reshape = False) 
    return image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Train

In [ ]:
for run in range(1, NR_OF_RUNS+1):

    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.20, shuffle= True)

    models = []
    accuracies = [0]
    patience = 0

    for i in count(1):

        print(f"\n ===== Train model {i} =====")

        # Set the seeds
        np.random.seed(run*i)
        tf.random.set_seed(run*i)

        # augmentation
        datagen = ImageDataGenerator(preprocessing_function=rotate_image)
        datagen = datagen.flow(x_train,y_train, batch_size= BATCH_SIZE)

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/hardhistory", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/hardweights", exist_ok=True)

        # Create the model
        model = CIFARmodel(IMAGE_SIZE, NUM_CLASSES, 3)
        
        # Load the weighs if the model is already trained
        weights_path = PATH + MODEL_NAME + f"/{run}/hardweights/weights-{i}.h5"

        if os.path.exists(weights_path):
            print(f"Skipping training of model {i}: weights exists")
            model.load_weights(weights_path)
        else:
            es = EarlyStopping(min_delta=0.01, patience=3)
            csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/hardhistory/history-{i}.csv", separator=';')

            model.fit_generator(datagen,
                                epochs = EPOCHS,
                                validation_data = (x_val, y_val),
                                shuffle = True,
                                callbacks=[es, csv_logger])
            
            model.save_weights(weights_path)
        
        models.append(model)

        acc = predict(models, x_val, y_val,voting='hard')
        delta = acc - accuracies[-1]

        accuracies.append(acc)

        if delta >= MODEL_ADDITION_DELTA:
          patience = 0
        else:
          patience += 1

        print(f"Model: {i} added. Resulting score: {acc}, Delta: {delta}, Patience: {patience}")

        if patience >= MODEL_ADDITION_PATIENCE:
          break
    print("\n ===== Accuracy vs nr of models =====")

    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df.insert(1, "Nr of models", accuracy_df.index)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy_{VOTING}.csv")
    display(accuracy_df)

    ## Accuracy
    ## The final accuracy of the ensamble on the test set
    print("\n ===== Accuracy ======")

    accuracy = predict(models, x_test, y_testc,voting='hard')
    print("Accuracy: " + str(accuracy))

    ## Correlation between models
    print("\n ===== Correlation =====")
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(x_test), axis=1))
    classified = []

    for prediction in predictions:
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
    correlation_matrix = []

    for ix, x in enumerate(classified):
      row = []
      
      for iy, y in enumerate(classified):
        if (ix == iy):
          row.append(np.nan)
        else:
          row.append(pearsonr(x,y)[0])

      correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix_{VOTING}.csv")
    
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))

    # Save the results
    file = PATH + MODEL_NAME + f"/results_{VOTING}.csv"
    df = pd.DataFrame([[run, accuracy, correlation]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy", "correlation"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)



 ===== Train model 1 =====
Epoch 1/9999
313/313 [==============================] - 17s 56ms/step - loss: 1.7167 - accuracy: 0.3913 - val_loss: 2.2135 - val_accuracy: 0.1691
Epoch 2/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.3475 - accuracy: 0.5227 - val_loss: 1.4767 - val_accuracy: 0.4850
Epoch 3/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.2013 - accuracy: 0.5776 - val_loss: 1.3490 - val_accuracy: 0.5258
Epoch 4/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.0954 - accuracy: 0.6155 - val_loss: 1.3330 - val_accuracy: 0.5285
Epoch 5/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.0108 - accuracy: 0.6475 - val_loss: 1.1237 - val_accuracy: 0.5997
Epoch 6/9999
313/313 [==============================] - 10s 33ms/step - loss: 0.9442 - accuracy: 0.6710 - val_loss: 1.1027 - val_accuracy: 0.6065
Epoch 7/9999
313/313 [==============================] - 10s 33ms/step - loss: 0.8839 - accuracy:

,Accuracy,Nr of models
0,0.0000,0
1,0.6481,1
2,0.6446,2
3,0.6820,3
4,0.6975,4
5,0.7036,5
6,0.7084,6
7,0.7154,7



 ===== Accuracy ======
0.7144
Accuracy: 0.7144

 ===== Correlation =====


,0,1,2,3,4,5,6
0,NaN,0.484698,0.390936,0.480342,0.401690,0.492977,0.454268
1,0.484698,NaN,0.411089,0.502908,0.451293,0.485122,0.498524
2,0.390936,0.411089,NaN,0.430256,0.448189,0.478688,0.481600
3,0.480342,0.502908,0.430256,NaN,0.495136,0.514495,0.488457
4,0.401690,0.451293,0.448189,0.495136,NaN,0.449628,0.499572
5,0.492977,0.485122,0.478688,0.514495,0.449628,NaN,0.489786
6,0.454268,0.498524,0.481600,0.488457,0.499572,0.489786,NaN


Average correlation: 0.4680787572166298

 ===== Train model 1 =====
Epoch 1/9999
313/313 [==============================] - 11s 35ms/step - loss: 1.6788 - accuracy: 0.4026 - val_loss: 2.2753 - val_accuracy: 0.2196
Epoch 2/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.3139 - accuracy: 0.5391 - val_loss: 1.4113 - val_accuracy: 0.5193
Epoch 3/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.1649 - accuracy: 0.5932 - val_loss: 1.1921 - val_accuracy: 0.5704
Epoch 4/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.0607 - accuracy: 0.6297 - val_loss: 1.1728 - val_accuracy: 0.5840
Epoch 5/9999
313/313 [==============================] - 10s 33ms/step - loss: 0.9840 - accuracy: 0.6589 - val_loss: 1.0578 - val_accuracy: 0.6285
Epoch 6/9999
313/313 [==============================] - 10s 33ms/step - loss: 0.9127 - accuracy: 0.6859 - val_loss: 1.1042 - val_accuracy: 0.6144
Epoch 7/9999
313/313 [==============================] - 

...

Average correlation: 0.4902435017645082

 ===== Train model 1 =====
Epoch 1/9999
313/313 [==============================] - 11s 36ms/step - loss: 1.7222 - accuracy: 0.3832 - val_loss: 2.1716 - val_accuracy: 0.1859
Epoch 2/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.3333 - accuracy: 0.5267 - val_loss: 1.4180 - val_accuracy: 0.4947
Epoch 3/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.1895 - accuracy: 0.5813 - val_loss: 1.2217 - val_accuracy: 0.5648
Epoch 4/9999
313/313 [==============================] - 10s 33ms/step - loss: 1.0807 - accuracy: 0.6215 - val_loss: 1.1666 - val_accuracy: 0.5790
Epoch 5/9999
313/313 [==============================] - 10s 33ms/step - loss: 0.9954 - accuracy: 0.6536 - val_loss: 1.1872 - val_accuracy: 0.5777
Epoch 6/9999
313/313 [==============================] - 10s 33ms/step - loss: 0.9269 - accuracy: 0.6783 - val_loss: 1.0546 - val_accuracy: 0.6227
Epoch 7/9999
313/313 [==============================] - 

,Accuracy,Nr of models
0,0.0000,0
1,0.6355,1
2,0.6629,2
3,0.6959,3
4,0.7099,4
5,0.7215,5
6,0.7249,6
7,0.7309,7
8,0.7304,8



 ===== Accuracy ======
0.7327
Accuracy: 0.7327

 ===== Correlation =====


,0,1,2,3,4,5,6,7
0,NaN,0.550197,0.490783,0.519872,0.443040,0.368078,0.501392,0.450055
1,0.550197,NaN,0.506270,0.534002,0.464540,0.400782,0.499172,0.497204
2,0.490783,0.506270,NaN,0.525309,0.489943,0.434037,0.492214,0.500040
3,0.519872,0.534002,0.525309,NaN,0.485591,0.431372,0.488065,0.499271
4,0.443040,0.464540,0.489943,0.485591,NaN,0.468243,0.477910,0.496108
5,0.368078,0.400782,0.434037,0.431372,0.468243,NaN,0.379039,0.484921
6,0.501392,0.499172,0.492214,0.488065,0.477910,0.379039,NaN,0.446580
7,0.450055,0.497204,0.500040,0.499271,0.496108,0.484921,0.446580,NaN


Average correlation: 0.47585822952013135


# Results

In [ ]:
from scipy import stats

In [ ]:
baseline_hard =  [0.7261,
0.7174,
0.7199,
0.7196,
0.7185,
0.7201,
0.7163,
0.7107,
0.72,
0.7199]

In [ ]:
np.mean(baseline_hard)

0.71885

In [ ]:
baseline_soft=   [0.7311,
0.7262,
0.7252,
0.7204,
0.7207,
0.7208,
0.7273,
0.7228,
0.7233,
0.7301]

In [ ]:
np.mean(baseline_soft)

0.7247899999999999

In [ ]:
augmented_hard = pd.read_csv(PATH+ "CIFAR_rotation/results_HARD.csv")
augmented_hard = augmented_hard.values
augmented_hard

array([[ 1.        ,  0.7326    ,  0.47983361],
       [ 2.        ,  0.7313    ,  0.47109997],
       [ 3.        ,  0.7203    ,  0.4783455 ],
       [ 4.        ,  0.728     ,  0.48827692],
       [ 5.        ,  0.7174    ,  0.47138347],
       [ 6.        ,  0.7188    ,  0.48603817],
       [ 7.        ,  0.7201    ,  0.47936342],
       [ 8.        ,  0.7258    ,  0.47636431],
       [ 9.        ,  0.725     ,  0.47767154],
       [10.        ,  0.7279    ,  0.46347953]])

In [ ]:
augmented_soft = pd.read_csv(PATH+"CIFAR_rotation/results_SOFT.csv")
augmented_soft = augmented_soft.values
augmented_soft

array([[ 1.        ,  0.7411    ,  0.47983361],
       [ 2.        ,  0.7389    ,  0.47109997],
       [ 3.        ,  0.7301    ,  0.4783455 ],
       [ 4.        ,  0.7385    ,  0.48827692],
       [ 5.        ,  0.7303    ,  0.47138347],
       [ 6.        ,  0.73      ,  0.48603817],
       [ 7.        ,  0.7286    ,  0.47936342],
       [ 8.        ,  0.7372    ,  0.47636431],
       [ 9.        ,  0.7338    ,  0.47767154],
       [10.        ,  0.7324    ,  0.46347953]])

In [ ]:
augmented_hard = augmented_hard[:,1]
augmented_soft = augmented_soft[:,1]

In [ ]:
np.mean(augmented_hard)

0.7247199999999999

In [ ]:
np.mean(augmented_soft)

0.7340899999999999

In [ ]:
stats.ttest_rel(augmented_hard,baseline_hard)

Ttest_relResult(statistic=3.236995628815524, pvalue=0.010207640998461252)

In [ ]:
stats.ttest_rel(augmented_soft,baseline_soft)

Ttest_relResult(statistic=5.586708076956365, pvalue=0.0003400133228561551)

## Accuracy
The final accuracy of the ensamble on the test set

In [ ]:
!zip -r /content/FashionMNIST_rotation.zip /content/FashionMNIST_rotation

  adding: content/FashionMNIST_rotation/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/accuracy_SOFT.csv (deflated 38%)
  adding: content/FashionMNIST_rotation/10/weights/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/weights/weights-1.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-5.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-2.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-6.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-4.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-3.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/history/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/history/history-4.csv (deflated 52%)
  adding: content/FashionMNIST_rotation/10/history/history-1.csv (deflated 52%)
  adding: content/FashionMNIST_rotation/10/history/hi